<a href="https://colab.research.google.com/github/chanseongparkk/2022_Data_Analysis_Intern_practice/blob/main/07_04_01_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%8B%A4%EC%88%98%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 범주형 데이터 실수화


In [ ]:
### 라이브러리 호출
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 데이터셋 생성
X_train = pd.DataFrame(["남성","여성","남성","여성","남성","여성"], columns = ["성별"])

# 데이터 확인
X_train.head(3)

,성별
0,남성
1,여성
2,남성


In [ ]:
X_train.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   성별      6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


In [ ]:
# 개체당 갯수 확인

X_train.성별.value_counts()

남성    3
여성    3
Name: 성별, dtype: int64

### 라벨인코딩

In [ ]:
# 라벨 인코더 생성
encoder = LabelEncoder()

# 피팅 후 변환
X_train["성별_인코딩"] = encoder.fit_transform(X_train["성별"])


In [ ]:
# 인코딩 결과 확인
print(X_train,"\n\n",
     list(encoder.classes_),"\n\n",   # 클래스 확인
     list(encoder.inverse_transform([1,0]))) # 인코딩 값으로 문자값 확인

   성별  성별_인코딩
0  남성       0
1  여성       1
2  남성       0
3  여성       1
4  남성       0
5  여성       1 

 ['남성', '여성'] 

 ['여성', '남성']


### 원핫인코딩

In [ ]:
X_train.head(3)

,성별,성별_인코딩
0,남성,0
1,여성,1
2,남성,0


In [ ]:
# 원핫인코더 생성
# sparse = True 설정할경우 행렬의 좌표리스트 형식, False의 경우 넘파이 배열로 반환

ohe = OneHotEncoder(sparse = False)

# X_train data를 이용하여 피팅

ohe.fit_transform(X_train[["성별"]])

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [ ]:
# 더미화된 feature 확인

print(ohe.get_feature_names())

['x0_남성' 'x0_여성']


## 텍스트  데이터 실수화

### 단어의 빈도수를 카운트하여 수치화
- CountVectorizer 함수: 문서집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 Bag of Words로 인코딩하는 함수

In [ ]:
# 라이브러리 임포트

from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# 데이터셋 생성
corpus = [
    '청년 인재 개발 양성 과정',
    '인공지능 청년 양성',
    '미래 인공지능 데이터 대한민국',
    '데이터 원유 기술사 청년 개발'
]

print(corpus)

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']


In [ ]:
type(corpus) #list로 저장됨

list

In [ ]:
### 텍스트 실수화

# 카운트 벡터라이저 생성
count_vect = CountVectorizer()

# 단어 카운트
count_vect.fit(corpus)

# 단어 확인
count_vect.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [ ]:
# 단어 벡터화한 값을 array로 변환하여 확인
count_vect.transform(corpus).toarray()

array([[1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]], dtype=int64)

In [ ]:
# 단어 벡터화

features = count_vect.transform(corpus)

features

<4x11 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [ ]:
### 문서단어행렬로 변환

# 속성 이름만 반환
vocab = count_vect.get_feature_names()

vocab

['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지능', '인재', '청년']

In [ ]:
DTM = pd.DataFrame(feature.toarray(),columns = vocab)

DTM

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지능,인재,청년
0,1,1,0,0,0,0,1,0,0,1,1
1,0,0,0,0,0,0,1,0,1,0,1
2,0,0,0,1,1,1,0,0,1,0,0
3,1,0,1,0,1,0,0,1,0,0,1


In [ ]:
#문서단어행렬에 일치하는 단어 확인
count_vect.transform(["기술사 대한민국 인재 만세"]).toarray()

array([[0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0]], dtype=int64)

### 단어의 빈도수를 단어들마다 중요한 정도를 가중치로 주어 수치화(TF-IDF score)
- TfidfVectorizer 함수: TF-IDF 방식으로 단어의 가중치를 조정한 BOW로 인코딩하는 방법

In [ ]:
### 라이브러리 호출

from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 데이터셋 확인
corpus

['청년 인재 개발 양성 과정', '인공지능 청년 양성', '미래 인공지능 데이터 대한민국', '데이터 원유 기술사 청년 개발']

In [ ]:
### 텍스트 실수화 :TF-IDF 방식

# TF-IDF 벡터라이저 생성
tf_idf = TfidfVectorizer()

# 단어 카운트
tf_idf.fit(corpus)

# 단어사전 확인
tf_idf.vocabulary_

{'청년': 10,
 '인재': 9,
 '개발': 0,
 '양성': 6,
 '과정': 1,
 '인공지능': 8,
 '미래': 5,
 '데이터': 4,
 '대한민국': 3,
 '원유': 7,
 '기술사': 2}

In [ ]:
# 단어 벡터화한 값을 array로 변환하여 확인
tf_idf.transform(corpus).toarray()

array([[0.41263976, 0.52338122, 0.        , 0.        , 0.        ,
        0.        , 0.41263976, 0.        , 0.        , 0.52338122,
        0.33406745],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.61366674, 0.        , 0.61366674, 0.        ,
        0.49681612],
       [0.        , 0.        , 0.        , 0.55528266, 0.43779123,
        0.55528266, 0.        , 0.        , 0.43779123, 0.        ,
        0.        ],
       [0.41263976, 0.        , 0.52338122, 0.        , 0.41263976,
        0.        , 0.        , 0.52338122, 0.        , 0.        ,
        0.33406745]])

In [ ]:
# 단어 벡터화
features_tfidf = tf_idf.transform(corpus)

features_tfidf

<4x11 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [ ]:
### 문서단어행렬로 변환

#속성 이름만 반환
vocab_tfidf = count_vect.get_feature_names()

vocab_tfidf

['개발', '과정', '기술사', '대한민국', '데이터', '미래', '양성', '원유', '인공지능', '인재', '청년']

In [ ]:
DTM_tfidf = pd.DataFrame(features_tfidf.toarray(),columns = vocab_tfidf)

DTM_tfidf

,개발,과정,기술사,대한민국,데이터,미래,양성,원유,인공지능,인재,청년
0,0.41264,0.523381,0.000000,0.000000,0.000000,0.000000,0.412640,0.000000,0.000000,0.523381,0.334067
1,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.613667,0.000000,0.613667,0.000000,0.496816
2,0.00000,0.000000,0.000000,0.555283,0.437791,0.555283,0.000000,0.000000,0.437791,0.000000,0.000000
3,0.41264,0.000000,0.523381,0.000000,0.412640,0.000000,0.000000,0.523381,0.000000,0.000000,0.334067
